In [1]:
import pandas as pd

## Warning - This block contains a somewhat expensive 'Groupby' operation that will take around 5 minutes

In [2]:
exsup = pd.read_csv('./sources/exsup.csv')
# Remove all non-coldwar entries
mask = exsup['ywp_year'] < 1992
exsup = exsup[mask]
#remove non US / SU interventions
dummies = exsup['external_name'].str.get_dummies(sep=';')
# 'Netherlands', 'Spain' and 'Portugal' are excluded because they are not in the Dataset as interveners.
dummies = dummies[['United States', 'Soviet Union', 'United Kingdom', 'France', 'Belgium']]
exsup = pd.concat([exsup, dummies], axis=1)
# Drop duplicates
exsup.drop_duplicates(inplace=True)

# Add support specific dummies
old = ['external_type_X', 'external_type_Y', 'external_type_L',
       'external_type_W', 'external_type_M', 'external_type_T',
       'external_type_$', 'external_type_I', 'external_type_O', 
       'external_type_U']
new = [
    'troops', 'joint_ops', 'terr', 'weapons', 'logistics',
    'training', 'eco', 'intel', 'other', 'unknown'
]

us_support_cols = ['US_' + x for x in old]
su_support_cols = ['SU_' + x for x in old]

for col in us_support_cols:
    exsup[col] = 0
for col in su_support_cols:
    exsup[col] = 0
   
for index, row in exsup.iterrows():
    if row['Soviet Union'] == 1:
        nation = 'SU_'
        for col in old:
            exsup[index: index + 1][nation + col] = int(row[col])
    if row['United States'] == 1:
        nation = 'US_'
        for col in old:
            exsup[index: index + 1][nation + col] = int(row[col])         

# Add (empty, since there are no values) dummies for Netherlands, Spain and Portugal
exsup['Netherlands'] = 0
exsup['Spain'] = 0
exsup['Portugal'] = 0

# Merge dataframe per year-country combination
exsup = exsup.groupby(['locationid1', 'ywp_year']).max()
exsup.to_csv('./output/EXSUP_PREPARED.csv')

/home/jasper/Dropbox/ULTRADOCUMENTEN/THESIS_CIVIL_WAR/THESIS/venv/lib/python3.5/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jasper/Dropbox/ULTRADOCUMENTEN/THESIS_CIVIL_WAR/THESIS/venv/lib/python3.5/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
